# Tutorial 2: Clusters and cluster correlations

## Building a pool of clusters

Now that you know how to build parent lattices and structures sets, the next task is to create a pool of clusters. A clusters pool can be used to define the basis set for the expansion of configuration-dependent properties in terms of cluster functions. Although this basis is infinite (equal in number to all symetrically distinct atomic configurations of the substituent species), in practical applications one must cut off the basis. In order to fix ideas, we will start with a very simple model (corresponding to the solution of **Excercise 1** of [Tutorial 1](https://sol.physik.hu-berlin.de/cell/tutorials/tutorial1.html)) of a binary two-dimensional square lattice. Afterwards, you will tackle the more complicated fcc(111) Al surface system of [Tutorial 1](https://sol.physik.hu-berlin.de/cell/tutorials/tutorial1.html), by solving **Excercise 4** of this tutorial.

To start, we must define the parent lattice:

In [27]:
from ase import Atoms
from clusterx.parent_lattice import ParentLattice
from clusterx.visualization import juview

a=4.0
pri = Atoms(positions=[[0,0,0]],cell=[[a,0,0],[0,a,0],[0,0,2*a]],pbc=(1,1,0)) # Atoms object. Only positions and cell are set

plat = ParentLattice(pri,site_symbols=[['Pd','Au']]) # ParentLattice object with assignement of species

juview(plat)


![Parent lattice 2D](fig2-1.png)


Now, we will create all possible clusters of up to four points and radius up to $\sqrt{2}\,a$. This is done with the following piece of code:

In [28]:
from clusterx.clusters.clusters_pool import ClustersPool

r = 1.42 # This is large enough to include all clusters up to sqrt(2)*a
cpool = ClustersPool(plat, npoints=[0,1,2,3,4], radii=[0,0,a*r,a*r,a*r])

In the first line, we load the [ClustersPool](https://sol.physik.hu-berlin.de/cell/modules/clusters_pool.html) class. Then, we create an object of this class, and call it `cpool`. To initialize it, we use the parameters `npoints` and `radii`. `npoints` indicates the number of points of the clusters in the pool, and the parameter `radii` indicates the maximum radius corresponding to each of the number of points indicated in `npoints`. In this way, we have created a clusters pool containing the empty cluster, all the 1-point clusters (in this case there will be only one of this kind), and all the 2-, 3-, and 4-point clusters up to radius $1.42\times a$.

The number of clusters just created is:

In [29]:
print("Number of clusters: ", len(cpool))

Number of clusters:  6


Now, we have at least two ways of visualizing the pool of clusters: *i)* we can plot all of them on this notebook with `juview` (or only a few, by using the parameter `n` of `juview`):

In [30]:
juview(cpool)


![Clusters pool](fig2-2.png)


or *ii)* by using the [graphical user interface (gui) of ASE](https://wiki.fysik.dtu.dk/ase/ase/gui/gui.html#module-ase.gui) on a terminal. To use this second option, which is the recommended way to proceed when the clusters pool is large, first create a clusters database with

In [31]:

cpool.write_clusters_db()

FileNotFoundError: [WinError 2] The system cannot find the file specified

The execution of this line will create a file called `cpool.json` in the same folder where this tutorial is run. This file has the proper format to be visualized with ASE's gui. Now, open a terminal and execute `$>ase gui cpool.json` (the `$>` denotes the command prompt). A number of windows will open. The relevant ones for the visualization of the clusters are shown in the screen capture below (if you can not visualize the image below, ):

![ASE GUI](asegui.png)

You can visualize all the clusters by clicking on the "Back" and "Forward" buttons of the "Movie" window on the right. Let's see how this works for a larger clusters pool:

In [11]:
r = 2.5

cpool = ClustersPool(plat, npoints=[0,1,2,3,4,5], radii=[0,0,a*r,a*r,a*r,a*r])
cpool.write_clusters_db()

print(len(cpool), " clusters were created.")

FileNotFoundError: [WinError 2] The system cannot find the file specified

Now, as an excercise, create a clusters pool database and visualize these 34 clusters with ASE's gui as explained above. You can also view a few of them in a new cell below by executing `juview` with a small parameter `n`.

In [7]:
juview(cpool)

## Cluster orbits

The clusters obtained above are all symmetrically inequivalent. Moreover, each of them is a representative of an infinite set of symmetrically equivalent clusters. Every infinite set is called a cluster orbit. We can calculate a cluster orbit for a supercell and visualize it with either `juview` or ASE's gui. Let's do so for one cluster of the clusters pool `cpool`:

In [8]:
# Obtain the orbit for cluster number 8
cluster_orbit, cluster_multiplicity = cpool.get_cluster_orbit(cluster_index = 8) 

print("There are ",len(cluster_orbit),"symmetrically equivalent representations of the cluster in the supercell.")
print("The cluster multiplicity is ",cluster_multiplicity,".")

C:\Users\Max\PycharmProjects\Project1\venv\lib\site-packages\clusterx\clusters\clusters_pool.py:508: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(atom_idxs), np.array(atom_nrs)


There are  100 symmetrically equivalent representations of the cluster in the supercell.
The cluster multiplicity is  4 .


The cluster multiplicity is the number of symetrically equivalent realizations of the cluster, under the symmetry operations of the parent cell, without counting the internal translations of the parent cell inside the supercell. 

Now, we create a clusters database containing the cluster orbit:

In [9]:
cluster_orbit.write_clusters_db(db_name="cluster_orbit.json")

FileNotFoundError: [WinError 2] The system cannot find the file specified

After this, you may visualize the cluster orbit by executing the command `$>ase gui cluster_orbit.json` in a terminal. Alternatively (without the need of executing the line above), you can use `juview`, as exemplified here by plotting the first 6 clusters of the orbit:

In [12]:
juview(cluster_orbit,n=6)

![Cluster orbit](fig2-3.png)

## Cluster correlations

In this section we will illustrate the calculation of the cluster correlations for the simple binary case studied here. To simplify the analysis, we will use a (non-orthogonal) basis-set consisting of the following site cluster functions:

\begin{equation*}
\theta_0(\sigma_i) = 1\\
\theta_1(\sigma_i) = \sigma_i.
\end{equation*}

Here $\sigma_i$ is the site occupation variable for the crystal site $i$, taking the value $\sigma_i=1$ if the crystal site $i$ is occupied with the substitutional species (Ge in this example), or the value $\sigma_i=0$ otherwise (Si in this example). Using this basis, one can define a basis set of *cluster functions* $\psi_\alpha(\sigma_{s})$, with $\sigma_{s}=(\sigma_{s1},\sigma_{s2},...,\sigma_{sN})$ a vector representing the configuration of the structure $s$, and $\alpha=\{\alpha_1,\alpha_2,...\}$ a set of crystal sites representing a cluster:

\begin{equation*}
\psi_\alpha(\sigma_{s})=\prod_{i\in\alpha}\theta_1(\sigma_{si})=\prod_{i\in\alpha}\sigma_{si}
\end{equation*}

According to this definition, $\psi_\alpha(\sigma_{s})=1$ if the substitutional pattern represented by cluster $\alpha$ is present in structure $s$, and $\psi_\alpha(\sigma_{s})=0$ otherwise. With this, the *cluster correlations* $X_{s\alpha}$ are obtained from the orbit-averaged cluster-functions:  

\begin{equation*}
X_{s\alpha}=\frac{1}{M_\alpha}\sum_{\beta\in{\cal O}({\alpha})}{\psi_\beta(\sigma_{s})} = \frac{M_{\alpha\in s}}{M_\alpha}
\end{equation*}

Here, ${\cal O}({\alpha})$ is the set of all clusters in the orbit of $\alpha$, $M_{\alpha\in s}$ the number of clusters from ${\cal O}({\alpha})$ which are present in structure $s$, and $M_\alpha$ is the total number of clusters in ${\cal O}({\alpha})$. Therefore, colloquially we can say that the correlation $X_{s\alpha}$ represents the concentration of the cluster-type $\alpha$ in structure $s$. Note that this interpretation is only valid in the case of binary alloys for the basis used in this example. The more general definitions of basis sets for multicomposition/multilattice alloys in CELL do not admit this interpretation.

Let's now calculate the cluster correlations for the same small pool of 6 clusters obtained at the start of this tutorial:

In [13]:
r = 1.42
cpool = ClustersPool(plat, npoints=[0,1,2,3,4], radii=[0,0,a*r,a*r,a*r])
juview(cpool)


![Cllusters pool](fig2-2.png)


To accomplish this task, we must first create a [CorrelationsCalculator](https://sol.physik.hu-berlin.de/cell/modules/correlations.html) object:

In [14]:
from clusterx.correlations import CorrelationsCalculator

corrcal = CorrelationsCalculator("binary-linear", plat, cpool)

By using the paramenter `"binary-linear"`, the correlations will be computed using the basis set explained above.

Let's now create a random structure and then obtain the cluster correlations for this structure:

In [19]:
from clusterx.super_cell import SuperCell

scell = SuperCell(plat,[5,5]) # Create a 4x4 supercell
structure = scell.gen_random({0:[3]}) # Random structure with 3 substitutional atoms 
juview(structure.get_atoms())

NGLWidget()

Now, we calculate the correlations for this structure and display some relevant information, namely, *i)* the cluster index $\alpha$, $ii)$ the length of the orbit for the cluster in the supercell ($M_\alpha$), and $iii)$ the correlation of cluster $\alpha$ to structure $s$ ($X_{s\alpha}$).

In [20]:
corrs = corrcal.get_cluster_correlations(structure) # Compute correlations for structure 
orbl = corrcal.get_orbit_lengths(structure) # Get orbit lengths in structure M_\alpha

print("{0:<19s}|{1:<19s}|{2:<19s}".format("Cluster index","Orbit length","Correlation"))
for i in range(len(cpool)):
    print("{0:<19d}|{1:<19d}|{2:<19.5f}".format(i, orbl[i], corrs[i]))

Cluster index      |Orbit length       |Correlation        
0                  |1                  |1.00000            
1                  |25                 |0.12000            
2                  |50                 |0.02000            
3                  |50                 |0.00000            
4                  |100                |0.00000            
5                  |25                 |0.00000            


## Excercise 3

a) Using the figures of the clusters, the figure of the random structure generated above, and the orbit lengths, calculate *by hand* the correlations for the random structure and verify that they are equal to what is returned by the `get_cluster_correlations()` method. (Note: the correlation of the *empty* cluster, i.e. that one with `npoints`=0, is always 1 by definition.)

b) Generate a new random structure with a different concentration, recalculate the correlations, and repeat item a).

c) To which familiar quantity can you relate the correlation for the 1-body cluster?

## Excercise 4

Create a pool of clusters for the surface system of Tutorial 1 and visulaize the generated clusters with ASE's gui